In [16]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import skimage
import os
import pickle

nAnchors = 24
batchSize = 16
numEpochs = 2000
device = "cpu"

In [17]:
class AnchorDataSet(torch.utils.data.Dataset):
    def __init__(self, file, transform = None, target_transform=None):
        super(AnchorDataSet, self).__init__()
        self.df = pd.read_pickle(file)
        self.transform = transform
        self.target_transform=target_transform
        """
        self.filenames = dummy.loc[:,"ImageFile"]
        self.features = dummy.loc[:,"Features"]
        self.anchorDists = dummy.loc[:,"AnchorDists"]
        self."""
    
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self,idx):
        
        feats = self.df.loc[idx, "Features"].astype(np.double)
        anchorDists = self.df.loc[idx, "AnchorDists"].astype(np.double)
        
        dofs = np.array([self.df.loc[idx,"Z"],self.df.loc[idx,"W"],self.df.loc[idx,"P"],self.df.loc[idx,"Q"],self.df.loc[idx,"R"]]).astype(np.double)
        
        xy = np.array([self.df.loc[idx,"X"], self.df.loc[idx,"Y"]]).astype(np.double)
        imageFile = self.df.loc[idx, "ImageFile"]
        ImageNum = self.df.loc[idx,"ImageNum"]

        return {"Features": feats,
                "anchorDists": anchorDists,
                "dofs": dofs,
                "xy": xy,
                "ImageFile": imageFile,
                "ImageNum": ImageNum}


class anchorNet(nn.Module):
    def __init__(self, nAnchors):
        super(anchorNet, self).__init__()
        self.classifier = nn.Linear(2208, nAnchors)
        self.regressor = nn.Linear(2208, 2*nAnchors)
        self.dof_regressor = nn.Linear(2208, 5)
        self.softmax = nn.Softmax(dim=1)
        self.double()
    
    def forward(self, feats):

        classify = self.softmax(self.classifier(nn.functional.relu(feats))) # TODO: ReLU useful?
        regress = self.regressor(nn.functional.relu(feats))
        dof_regress = self.dof_regressor(nn.functional.relu(feats))

        return classify, regress, dof_regress

def custom_loss(classify, regress, dof_regress, anchorDistsGt, dofGt, dofLoss,factors = [2.4,0,0.5]):
    # TODO: Possible Flaw (or the reason why this works?):
    # The net tries to learn realtive position to each anchorpoint
    # independently, so we have a large amount of degrees of freedom, even though relative
    # position to anchor points should have 2 DOF. Maybe try using more anchorpoints to
    # proof this point.

    """
    classify: output of anchor classifier
    regress: output of regressor
    dof_regress: output of dof_regressor
    anchorDistsGt: true distance to all anchor poitns
    dofGt: true remaining 4 DOF
    dofLoss: loss function used for dof_regressor
    factors: list of hyperparameters for weighting of different loss terms
    """

    dist = (regress.reshape(-1, nAnchors, 2) - anchorDistsGt)

    # TODO: Use softmax on gt-dof and dof for normalization
    lossXY = torch.sum(torch.linalg.norm(dist, axis = -1) * classify)
    # TODO: implement cross entropy
    
    return torch.sum(factors[0] * lossXY) + factors[1] + factors[2] * dofLoss(dof_regress, dofGt)


In [18]:
trainDataset = AnchorDataSet(file= "./ShopFacade/traindata_with_features_and_anchors.pkl")
trainDataloader = torch.utils.data.DataLoader(trainDataset, batch_size=16, shuffle=False, num_workers=0)

myNet = anchorNet(nAnchors=nAnchors).to(device)
dofLoss = nn.MSELoss()

optimizer = torch.optim.Adam(myNet.parameters(), lr=0.0003)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)

myAnchors = torch.from_numpy(np.loadtxt(f"./ShopFacade/anchors{nAnchors}.txt"))

for epoch in range(numEpochs):
    epochLoss = 0
    trainDists = []
    for i, data in enumerate(trainDataloader):

        myNet.train()

        optimizer.zero_grad()
        
        classify, regress, dof_regress = myNet.forward(data["Features"])
        loss = custom_loss(classify, regress, dof_regress, anchorDistsGt=data["anchorDists"], dofGt=data["dofs"], dofLoss=dofLoss)
        loss.backward()
        
        optimizer.step()
        epochLoss += loss.data.item()
        with torch.no_grad():
            bestAnchor = torch.argmax(classify, axis=1)

            dist = 0

            for j in range(bestAnchor.size()[0]):
                dist += torch.linalg.norm(torch.abs((myAnchors[bestAnchor[j], :] + regress.reshape(-1,nAnchors,2)[j,bestAnchor[j], :]) - (data["xy"])[j]))

            trainDists.append(dist.data.item() / bestAnchor.size()[0])

    if epoch%400==0:
        print(f"Loss in epoch {epoch}: ", epochLoss)
        print("Training Dist: ", np.mean(trainDists))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument mat1 in method wrapper_addmm)